In [1]:
import h5py
import torch
from selected_system import mols, mol_file
from ani_benchmark import ANIBenchmark
import pandas
import os
import tqdm
from IPython.display import display
import itertools
tqdm.monitor_interval = 0

def pretty_print(orig_dict):
    """Pretty print times in a dictionary"""
    ret = {}
    for i in orig_dict:
        if orig_dict[i] is None:
            ret[i] = None
        else:
            ms = int(orig_dict[i])
            s = ms / 1000
            m = s / 60
            h = m / 60
            if ms < 1000:
                p = '{}ms'.format(ms)
            elif s < 60:
                p = '{:.2f}s'.format(s)
            elif m < 60:
                p = '{:.2f}min'.format(m)
            else:
                p = '{:.2f}min'.format(h)
            ret[i] = p
    return ret

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


van Der Waals correction will be unavailable. Please install ased3


In [2]:
torch.set_num_threads(1)
fm = h5py.File(os.path.join('../',mol_file), "r")

benchmarks = {
    'C': ANIBenchmark(device=torch.device("cpu")),
}

if torch.cuda.is_available():
    benchmarks.update({
        'G': ANIBenchmark(device=torch.device("cuda")),
    })

In [3]:
for i in mols:
    print('number of atoms:', i)
    smiles = mols[i]
    for s in smiles:
        print('Running benchmark on molecule', s)
        key = s.replace('/', '_')
        coordinates = torch.from_numpy(fm[key][()])
        coordinates = coordinates[:200]
        species = fm[key].attrs['species'].split()
        results = {}
        for b,m in tqdm.tqdm_notebook(list(itertools.product(benchmarks, ['1','B']))):
            bench = benchmarks[b]
            coordinates = coordinates.type(bench.aev_computer.dtype)
            try:
                if m == '1':
                    result = bench.oneByOne(coordinates, species)
                elif m == 'B':
                    result = bench.inBatch(coordinates, species)
                else:
                    raise ValueError('BUG here')
                result['forward'] = result['aev'] + result['energy'] + result['neighborlist']
                result['total'] = result['forward'] + result['force']
            except RuntimeError as e:
                print(e)
                result = {'aev': None, 'energy': None, 'force': None, 'total': None, 'neighborlist': None }
            results[b + ',' + m] = pretty_print(result)
        df = pandas.DataFrame(results)
        display(df)
        break

number of atoms: 20
Running benchmark on molecule COC(=O)c1ccc([N+](=O)[O-])cc1


A Jupyter Widget

,"C,1","C,B","G,1","G,B"
aev,8.14s,391ms,21.39s,154ms
energy,653ms,28ms,1.57s,7ms
force,9.37s,864ms,30.65s,193ms
forward,9.20s,424ms,25.62s,173ms
neighborlist,408ms,4ms,2.65s,11ms
total,18.57s,1.29s,56.27s,367ms


number of atoms: 50
Running benchmark on molecule O=[N+]([O-])c1ccc(NN=Cc2ccc(C=NNc3ccc([N+](=O)[O-])cc3[N+](=O)[O-])cc2)c([N+](=O)[O-])c1


A Jupyter Widget

...

A Jupyter Widget

,"C,1","C,B","G,1","G,B"
aev,55.33s,8.82s,2.48min,1.74s
energy,2.85s,126ms,7.59s,40ms
force,1.45min,22.75s,3.59min,2.24s
forward,1.00min,9.03s,2.84min,1.85s
neighborlist,2.02s,78ms,14.11s,69ms
total,2.45min,31.77s,6.43min,4.09s


number of atoms: 305
Running benchmark on molecule [H]/N=C(/N)\NCCC[C@H](C(=O)N[C@H]([C@@H](C)O)C(=O)N[C@H](Cc1ccc(cc1)O)C(=O)NCCCC[C@@H](C(=O)NCCCC[C@@H](C(=O)NCC(=O)O)NC(=O)[C@H](CCCCNC(=O)[C@@H](Cc2ccc(cc2)O)NC(=O)[C@@H]([C@@H](C)O)NC(=O)[C@@H](CCCN/C(=N\[H])/N)N)NC(=O)[C@@H](Cc3ccc(cc3)O)NC(=O)[C@@H]([C@@H](C)O)NC(=O)[C@@H](CCCN/C(=N\[H])/N)N)NC(=O)[C@@H](Cc4ccc(cc4)O)NC(=O)[C@@H]([C@@H](C)O)NC(=O)[C@@H](CCCN/C(=N\[H])/N)N)N


A Jupyter Widget

,"C,1","C,B","G,1","G,B"
aev,2.87min,25.13s,7.85min,9.33s
energy,8.54s,416ms,23.94s,212ms
force,5.27min,3.33min,12.71min,11.43s
forward,3.14min,26.27s,9.05min,10.07s
neighborlist,7.35s,721ms,48.06s,523ms
total,8.40min,3.76min,21.76min,21.49s
